<a href="https://colab.research.google.com/github/navrat/EVA_phase1_2022_23/blob/main/S7_CIFAR_RESNET_GRADCAM/S7_RESNET_CIFAR_GRADCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
autoreload 2
# Future print function
from __future__ import print_function

# Load modules/models from EVA course
!git clone https://github.com/sujitojha1/pytorch-eva8 -q --quiet
!mv pytorch-eva8/* .
print("Loaded EVA8 Modules and Models")

!rm -rf pytorch-eva8

# Installing latest Albumentation library
!pip install -U git+https://github.com/albu/albumentations -q --quiet

# For inline matplotlib plotting
%matplotlib inline

In [ ]:
from utils import *

# Loading test and train data & dataloader
cifar10 = dataset_cifar10(batch_size=128)

# Train dataloader
trainloader = cifar10.loader(train_flag=True)

# Test dataloader
testloader = cifar10.loader(train_flag=False)

classes = cifar10.classes

In [ ]:
cifar10.data_summary_stats()
#print(vars(train))

In [ ]:
from models.resnet import *
import torch

# Loading ResNet18 Model
net = ResNet18()

# Using cuda if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

# Sending network to device
net = net.to(device)

In [ ]:
!pip install torchsummary
from torchsummary import summary

# Model Summary
summary(net, input_size=(3, 32, 32))

In [ ]:
# Load train and test model
from main import trainNetwork

trainObj, testObj = trainNetwork(net, device, trainloader, testloader, EPOCHS=20, lr=0.01)

In [ ]:
from utils import plot_loss_accuracy_graph
plot_loss_accuracy_graph(trainObj, testObj,EPOCHS=20)

In [ ]:
def calClassAccuracy(net, dataloader, classes, device):
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images,labels = images.to(device),labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
from utils import plot_misclassified_images
plot_misclassified_images(net, testloader, classes, device)

In [ ]:
from utils import plotGradCAM
# initialize a model, model_dict and gradcam
plotGradCAM(net, testloader, classes, device)

##END